#### 해당 파일은 Python ver 3.7.15으로 만들었습니다.

#### conda환경(visual studio code포함)이면 anaconda powershell prompt을 실행시키고
#### conda create -n 환경이름 python=3.7
#### 환경이름만 영문으로 하고싶은거 아무거나 작성하시고 실행하면 됩니다.
#### 이후 visual studio code들어와서 python환경을 방금 만든 환경으로 바꾸고 아래 파일 실행시키면 됩니다
#### path = 부분만 자신에 맞게 수정하면 아래코드는 에러없이 돌아갈겁니다.

In [45]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False


import warnings
warnings.filterwarnings('ignore')

In [87]:
# 파일이 있는 주소로 맞춰서 부르기
# 이거만 수정하면 됩니다.
path = 'C:/Users/User/github/Quant/data' 

'''

여기부터는 코드 안바꿔도 실행됩니다.
! pip install PyPortfolioOpt
! pip install cvxopt
만약 실행 안되면 위의 두개 코드 실행시키면 됩니다.
다운로드 안되면 python 버전 확인하거나

conda install PyPortfolioOpt
conda install cvxopt
두개중에 다운로드 실패한 라이브러리 다운로드 하고 실행해보세요.

'''

df = pd.read_csv(path + "/kospi_category.csv", encoding='cp949') 
start_date = '2004-01-01'
end_date = '2020-03-01'
 
df.index = pd.date_range(start_date,end_date,freq='m')
df.drop('date',axis=1, inplace=True)

In [75]:
# train 데이터와 validation 데이터 나누기.
X_train = df[df.index < '2019-01-01']
X_valid = df[df.index >= '2019-01-01']

# 각 데이터의 사이즈 확인하기
print('X_train Shape', X_train.shape)
print('X_Valid Shape', X_valid.shape)

X_train Shape (180, 22)
X_Valid Shape (14, 22)


In [76]:
import cvxopt as opt # python version 3.6-3.10 가능
from cvxopt import solvers
from pypfopt.efficient_frontier import EfficientFrontier # 정확하게는 모르겠는데 3.7은 사용가능
from pypfopt import risk_models
from pypfopt import expected_returns

In [77]:
weights = np.ones(22) # 초기에 가중치를 1로 설정

In [78]:
weights = weights/22 # 이후 동일하게 1/22으로 나눠서 설정 (가중치 합은 항상 1)

In [88]:
returns=X_train.pct_change() # 단순 수익률 계산 => (다음날 값 +- 기준일 값)/기준일 값 으로 수익률 나옵니다.
returns

,제조업,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,전기전자,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,은행,증권,보험,서비스업
2004-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-02-29,0.036156,-0.021484,-0.006045,0.126518,0.083113,0.074691,-0.021670,0.097743,0.086890,0.033444,...,-0.003526,0.027870,0.042927,-0.028853,0.044388,0.077803,0.063243,0.048602,0.044860,0.036380
2004-03-31,0.020653,-0.011740,-0.049002,0.005074,-0.004730,-0.021993,-0.029926,-0.057165,0.012194,0.056788,...,-0.031618,-0.016719,0.001519,-0.059017,-0.070340,-0.048453,-0.076715,-0.050630,0.050562,0.056952
2004-04-30,-0.027754,0.060707,0.007120,-0.082229,0.025673,0.029165,0.035643,-0.095249,-0.098894,-0.026716,...,0.170097,-0.012699,0.007583,-0.097207,-0.055064,-0.036767,-0.025705,-0.063589,0.013168,0.071245
2004-05-31,-0.080380,-0.087235,-0.090702,-0.105040,-0.042056,-0.102412,-0.119328,-0.052510,-0.103163,-0.095124,...,0.049405,-0.030267,-0.107315,-0.034113,-0.029152,-0.088004,-0.059883,-0.152506,-0.074637,-0.040962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,0.023656,-0.062041,0.095574,0.150438,-0.008875,0.112697,0.092990,-0.004385,0.024792,0.030001,...,-0.004379,-0.080536,0.009747,-0.038653,0.040464,-0.014923,-0.038900,0.021139,-0.041110,0.024218
2018-09-30,-0.010227,0.003352,-0.022718,-0.014482,-0.004329,0.088796,0.055573,-0.033430,0.055455,-0.050406,...,0.052215,-0.000307,0.039353,0.029819,0.081804,0.049420,0.028784,0.035789,0.063313,0.000510
2018-10-31,-0.141753,-0.101044,-0.196182,-0.197683,-0.170362,-0.269583,-0.211780,-0.157192,-0.203769,-0.092563,...,-0.163821,-0.090159,-0.227893,-0.095636,-0.069906,-0.106503,-0.049108,-0.171327,-0.054521,-0.134691
2018-11-30,0.018889,0.080839,0.010262,0.001424,0.051888,0.047238,0.119739,0.014037,0.046933,-0.006691,...,0.044166,0.067360,0.122340,0.080446,0.080527,0.022741,-0.004877,0.113610,-0.032360,0.108176


In [80]:
cov_matrix_annual=returns.cov()*12 # 데이터 기준이 DAY가 아니라 MONTH이므로 12를 곱해 연간 수익률로 계산
cov_matrix_annual

,제조업,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,전기전자,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,은행,증권,보험,서비스업
제조업,0.034126,0.019964,0.026510,0.028660,0.033778,0.024925,0.031472,0.032716,0.045543,0.032883,...,0.028880,0.014083,0.042271,0.032839,0.008393,0.032855,0.029961,0.051383,0.025798,0.028937
음식료품,0.019964,0.036622,0.029421,0.023764,0.027701,0.032107,0.030606,0.023065,0.031923,0.009311,...,0.027817,0.018425,0.033359,0.029223,0.008055,0.025431,0.022022,0.038818,0.022798,0.026266
섬유의복,0.026510,0.029421,0.063482,0.036923,0.029685,0.034545,0.039460,0.032205,0.041923,0.016450,...,0.037336,0.018965,0.043512,0.036145,0.006925,0.032959,0.034117,0.047261,0.023624,0.030459
종이목재,0.028660,0.023764,0.036923,0.060666,0.030272,0.036456,0.045103,0.033919,0.044325,0.021875,...,0.029331,0.017205,0.048698,0.031520,0.010028,0.034442,0.032871,0.055733,0.021910,0.027963
화학,0.033778,0.027701,0.029685,0.030272,0.052497,0.034071,0.039289,0.039907,0.052690,0.018597,...,0.033925,0.016297,0.051608,0.040734,0.007764,0.034047,0.029586,0.053114,0.029195,0.038072
의약품,0.024925,0.032107,0.034545,0.036456,0.034071,0.084872,0.039372,0.027884,0.044204,0.012713,...,0.028488,0.019108,0.043643,0.032271,0.009991,0.030919,0.024296,0.052922,0.027874,0.031046
비금속광물,0.031472,0.030606,0.039460,0.045103,0.039289,0.039372,0.079333,0.039670,0.051820,0.018920,...,0.035127,0.020102,0.061191,0.036931,0.014625,0.035300,0.031758,0.058789,0.026234,0.032433
철강금속,0.032716,0.023065,0.032205,0.033919,0.039907,0.027884,0.039670,0.065218,0.049730,0.019418,...,0.032156,0.022793,0.050486,0.043870,0.011422,0.037866,0.037508,0.054610,0.028146,0.030004
기계,0.045543,0.031923,0.041923,0.044325,0.052690,0.044204,0.051820,0.049730,0.105306,0.029224,...,0.045888,0.018214,0.076234,0.060639,0.011383,0.046844,0.037783,0.078609,0.041377,0.046473
전기전자,0.032883,0.009311,0.016450,0.021875,0.018597,0.012713,0.018920,0.019418,0.029224,0.051108,...,0.018452,0.010551,0.026014,0.018503,0.007918,0.027912,0.025764,0.043765,0.019598,0.018929


In [81]:
port_variance=np.dot(weights.T, np.dot(cov_matrix_annual, weights)) # 포트폴리오 분산의 기대값 = WT * (공분산{= 시그마}) * W를 코드로 옆처럼 나타냄
port_variance

0.03494252101847688

In [82]:
port_volatility=np.sqrt(port_variance) # 포트폴리오 변동성의 기대값 = SQRT(WT * 공분산(=시그마) * W) // 그냥 표준편차 = 변동성
port_volatility

0.18692918717652648

In [83]:
portfolioSimpleAnnualReturn=np.sum(returns.mean()*weights)*12 # 연간 수익률 계산 DAY가 아니라 MONTH기준이므로 12곱하면 됨
portfolioSimpleAnnualReturn

0.09431622448724322

In [84]:
#expected annual return, volatility(risk), variance

percent_var=str(round(port_variance,2)*100)+'%'
percent_vol=str(round(port_volatility,2)*100)+'%'
percent_ret=str(round(portfolioSimpleAnnualReturn,2)*100)+'%'

print('Expected Annual Return :'+ percent_ret )
print('Annual Volatility/risk :'+ percent_vol )
print('Annual Variance :'+ percent_var )

'''
동일한 WEIGHT기준으로는 연간 수익률이 9%정도로 보여짐.
그런데 위험성은 19%임...
아래서 포트폴리오 최적화 진행할 것
'''

Expected Annual Return :9.0%
Annual Volatility/risk :19.0%
Annual Variance :3.0%


In [92]:
risk_models.sample_cov # parameter 확인

<function pypfopt.risk_models.sample_cov(prices, returns_data=False, frequency=252, log_returns=False, **kwargs)>

In [90]:
expected_returns.mean_historical_return # parameter 확인

<function pypfopt.expected_returns.mean_historical_return(prices, returns_data=False, compounding=True, frequency=252, log_returns=False)>

In [93]:
# Portfolio optimization

# Calculate the expexted returns and annualised sample covariance matrix of the asset returns
mu=expected_returns.mean_historical_return(X_train, frequency=12) # 앞과 동일하게 data가 month기준
S=risk_models.sample_cov(X_train, frequency= 12) # 앞과 동일하게 data가 month기준

# Optimize the max sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe() #Maximize the Sharpe ratio, and get the raw weights
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) #Note the weights may have some rounding error, meaning they may not add up exactly to 1 but should be close
ef.portfolio_performance(verbose=True)

OrderedDict([('제조업', 0.0), ('음식료품', 0.0), ('섬유의복', 0.0), ('종이목재', 0.0), ('화학', 0.26938), ('의약품', 0.55999), ('비금속광물', 0.0), ('철강금속', 0.0), ('기계', 0.0), ('전기전자', 0.16066), ('의료정밀', 0.00997), ('운수장비', 0.0), ('유통업', 0.0), ('전기가스업', 0.0), ('건설업', 0.0), ('운수창고업', 0.0), ('통신업', 0.0), ('금융업', 0.0), ('은행', 0.0), ('증권', 0.0), ('보험', 0.0), ('서비스업', 0.0)])
Expected annual return: 13.5%
Annual volatility: 21.6%
Sharpe Ratio: 0.53


(0.13492968879251077, 0.2161293716058797, 0.5317633967959224)

### 최적화 결과
#### 화학 0.26938
#### 전기전자 0.16066
#### 의료정밀 0.00997
#### 의약품 0.55999
#### 만큼 구매하면됨
#### 예상 수익률은 13.5%, 위험성은 21.6% 으로 기존보다 4.5% 개선된 효과를 가지고 있음. but 위험성은 2.6% 증가

#### 아래서 가격을 주어지면 몇 주 만큼 구매하는지 확인

In [99]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(X_train)
weights = cleaned_weights 

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000000000)
allocation, leftover = da.lp_portfolio()

print("Discrete allocation:", allocation) # 해당주를 얼마나 구입하는가
print("Funds remaining: ${:.2f}".format(leftover)) # 예상 수익률

Discrete allocation: {'화학': 55258229, '의약품': 48164181, '전기전자': 11120633, '의료정밀': 4672324}
Funds remaining: $902.32
